# Electrónica. PID

En este notebook se presenta un ejemplo de uso de los widgets con Python, aplicado a la electrónica. Concretamente se añade un Controlador PID a un sistema para mejorar la respuesta ante un escalón.

![](PID.png) 

A continuación, se muestra el código del programa que añade un controlador PID al sistema, grafica las variables, y muestra los sliders para dar al usuario la capacidad de realizar su propio experimento. **Ejecute** las siguientes celdas.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import ipywidgets as widgets
from IPython.display import display

In [ ]:
n = 100             # Número de puntos a representar
tf = 25.0           # Tiempo final
SP_start = 2.0      # Tiempo hasta que cambie la consigna

def planta(y,t,u):
    retardo = 1.0  # Retraso de la señal respecto a la consigna
    if t<(retardo+SP_start):
        dydt = 0.0  
    else:
        dydt = (2/7) * (5 * u - 2 * y + 3.75)
    return dydt

def pidPlot(Kc,Ti,Td):
    t = np.linspace(0,tf,n)                 # Crea vector de tiempo --> linspace(inicio, final, n_puntos)
    P= np.zeros(n)                          # Inicializa el término Porporcional
    I = np.zeros(n)                         # Inicializa el término Integral
    D = np.zeros(n)                         # Inicializa el término Derivativo
    e = np.zeros(n)                         # Inicializa el error
    C_out = np.zeros(n)                     # Inicializa la salida del Controlador PID
    PV = np.zeros(n)                        # Inicializa la variable del proceso
    SP = np.zeros(n)                        # Inicializa consigna (Set Point)
    SP_init = int(n/10)                     # Inicio del escalón
    SP[0:SP_init] = 0.0                     # Antes de escalón
    SP[SP_init:n] = 5.0                     # Después de escalón
    y0 = 0.0                                # Condición inicial
    
    # Se halla la salida en cada iteración
    for i in range(1,n):
        # Se simula el proceso para cada intervalo de tiempo
        ts = [t[i-1],t[i]]                           # Intervalo de tiempo
        y = odeint(planta,y0,ts,args=(C_out[i-1],))  # Halla la salida
        y0 = y[1]                                    # Guarda la salida para la siguiente condición inicial
        # Calcula la nueva salida del controlador C_out
        PV[i] = y[1]                                 # Actualiza la variable del proceso (realimentación = 1)
        e[i] = SP[i] - PV[i]                         # Calcula el error = SP - PV
        dt = t[i] - t[i-1]                           # Aproxima dt por su variación dt≂Δt=tf-ti
        P[i] = Kc * e[i]                             # Calcula el término Porporcional
        I[i] = I[i-1] + (Kc/Ti) * e[i] * dt          # Calcula el término Integral
        D[i] = -Kc * Td * (e[i]-e[i-1])/dt           # Calcula el término Derivativo
        C_out[i] = P[i] + I[i] + D[i]                # Calcula la nueva salida del controlador PID
        
    # Representación gráfica de las variables
    plt.figure(1,figsize=(15,10))
    plt.subplot(2,2,1)
    plt.plot(t,SP,'k-',linewidth=2,label='Consigna (SP)')
    plt.plot(t,PV,'r:',linewidth=2,label='Variable del proceso (PV)')
    plt.legend(loc='best')
    plt.grid()
    plt.xlabel('t')
    plt.subplot(2,2,2)
    plt.plot(t,P,'g-',linewidth=2,label=r'Proporcional = $K_c \; e(t)$')
    plt.plot(t,I,'b-',linewidth=2,label=r'Integral = $\frac{K_c}{T_i} \int_{i=0}^{t} e(t) \; dt $')
    plt.plot(t,D,'r--',linewidth=2,label=r'Derivativo = $-K_c T_d \frac{de(t)}{dt}$')    
    plt.legend(loc='best')
    plt.grid()
    plt.xlabel('t')
    plt.subplot(2,2,3)
    plt.plot(t,e,'m--',linewidth=2,label='Error (SP-PV)')
    plt.legend(loc='best')
    plt.grid()
    plt.xlabel('t')
    plt.subplot(2,2,4)
    plt.plot(t,C_out,'b--',linewidth=2,label='Salida del PID (C_out)')
    plt.legend(loc='best')
    plt.grid()
    plt.xlabel('t')
    plt.show()
    
Kc_slider = widgets.FloatSlider(value=0.0,min=-0.25,max=7.0,step=0.05)
Ti_slider = widgets.FloatSlider(value=4.0,min=0.01,max=7.0,step=0.1)
Td_slider = widgets.FloatSlider(value=0.0,min=0.0,max=2.0,step=0.1)

widgets.interact(pidPlot, Kc=Kc_slider, Ti=Ti_slider, Td=Td_slider);

**¡Modifica los parámetros a tu gusto para crear tu propio PID!** Si lo necesitas, puedes modifica la planta para estabilizar tu propio sistema.